[Reference](https://medium.com/@ccpythonprogramming/web-scraping-with-python-an-example-using-sqlite-c23912e96336)

In [2]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

# URL of the job listings page
URL = "https://example.com/jobs/data-scientist"

# SQLite database file
DB_FILE = "jobs.db"

def create_database():
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS job_listings
                 (id INTEGER PRIMARY KEY, title TEXT, company TEXT, location TEXT, date_posted TEXT, url TEXT)''')
    conn.commit()
    conn.close()

def scrape_jobs():
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')

    jobs = []
    listings = soup.find_all('div', class_='job-listing')

    for listing in listings:
        title = listing.find('h2', class_='job-title').text.strip()
        company = listing.find('div', class_='company').text.strip()
        location = listing.find('div', class_='location').text.strip()
        date_posted = listing.find('div', class_='date-posted').text.strip()
        job_url = listing.find('a', class_='apply-link')['href']

        jobs.append((title, company, location, date_posted, job_url))

    return jobs

def save_jobs_to_db(jobs):
    conn = sqlite3.connect(DB_FILE)
    c = conn.cursor()
    c.executemany('INSERT INTO job_listings (title, company, location, date_posted, url) VALUES (?, ?, ?, ?, ?)', jobs)
    conn.commit()
    conn.close()

if __name__ == "__main__":
    create_database()
    while True:
        jobs = scrape_jobs()
        save_jobs_to_db(jobs)
        print(f"{len(jobs)} jobs scraped and saved to database.")
        time.sleep(3600)  # Wait for 1 hour before scraping again